# UW YAML Tools and Techniques

Set up our `PATH` so we can use the `demo` script:

In [1]:
import os
d = "../bin"
if d not in os.environ["PATH"].split(":"):
    os.environ["PATH"] += f":{d}"

## Native YAML Anchor and Alias

Use `&` to define an anchor and `*` to reference the anchored data.

Reference a value:

In [2]:
!realize tt00

tt00.yaml:

pi: &pi 3.142
constants:
  e: 2.718
  pi: *pi

+ uw config realize -i ../configs/tt00.yaml
pi: 3.142
constants:
  e: 2.718
  pi: 3.142


Merge mappings:

In [3]:
!realize tt01.yaml

tt01.yaml.yaml:

cat: ../configs/tt01.yaml.yaml: No such file or directory

+ uw config realize -i ../configs/tt01.yaml.yaml
Traceback (most recent call last):
  File "/tmp/pm/conda/envs/DEV-uwtools/bin/uw", line 7, in <module>
    sys.exit(main())
  File "/home/Paul.Madden/git/uwtools/src/uwtools/cli.py", line 100, in main
    sys.exit(0 if modes[args[STR.mode]](args) else 1)
  File "/home/Paul.Madden/git/uwtools/src/uwtools/cli.py", line 226, in _dispatch_config
    return actions[args[STR.action]](args)
  File "/home/Paul.Madden/git/uwtools/src/uwtools/cli.py", line 265, in _dispatch_config_realize
    uwtools.api.config.realize(
  File "/home/Paul.Madden/git/uwtools/src/uwtools/api/config.py", line 149, in realize
    return _realize(
  File "/home/Paul.Madden/git/uwtools/src/uwtools/config/tools.py", line 83, in realize
    input_obj = _realize_input_setup(input_config, input_format)
  File "/home/Paul.Madden/git/uwtools/src/uwtools/config/tools.py", line 180, in _realize_input_se

But merging mappings is shallow, and can cause problems:

In [4]:
!realize tt02

tt02.yaml:

defaults: &defaults
  10m_u_component_of_wind:
    level_type: heightAboveGround
    name: u_10m
gfs:
  <<: *defaults
  10m_u_component_of_wind:
    name: UGRD

+ uw config realize -i ../configs/tt02.yaml
defaults:
  10m_u_component_of_wind:
    level_type: heightAboveGround
    name: u_10m
gfs:
  10m_u_component_of_wind:
    name: UGRD


The `10m_u_component_of_wind` value under `gfs` completely replaced the value from `defaults` instead of just updating the name. This is something that `uw config realize` and `uw config compose` can help with, by doing a deep merge that treats every level independently.

## Jinja2 Expressions

The [Jinja2 templating language](https://jinja.palletsprojects.com/en/stable/intro/) lends significant power to the `uwtools` config and `template` modes, supporting hierarchical variable references, computation of values, generation of content with loops, and transformation of values with filters.

Here, `uwtools` empowers Jinja2 to refer to values within the YAML document to compute the number of points in a 3D data cube:

In [5]:
!realize tt03

tt03.yaml:

const:
  nx: 1440
  ny: 721
var:
  nlev: 127
  points: '{{ const.nx * const.ny * nlev }}'

+ uw config realize -i ../configs/tt03.yaml
const:
  nx: 1440
  ny: 721
var:
  nlev: 127
  points: '131856480'


- `const.nx` and `const.ny` are absolute references, from the top of the config.
- `nlev` is a reative (sibling) reference.
- The Jinja2 `{{ ... }}` syntax must be quoted to avoid conflicts with YAML syntax, which leads the computed `points` value being a string. We'll fix this soon.

Jinja2 loops can be used to compute compound values:

In [6]:
!realize tt04

tt04.yaml:

leadtimes: '{% for n in range(6, 25, 6) %}{{ "%03d" % n }},{% endfor %}'

+ uw config realize -i ../configs/tt04.yaml
leadtimes: 006,012,018,024,


Here, `leadtimes` is again a string value, and we'll do something more useful with it soon.

Finally, Jinja2 provides various [built-in filters](https://jinja.palletsprojects.com/en/stable/templates/#builtin-filters) to transform values:

In [7]:
!realize tt05

tt05.yaml:

model_name: FV3
variable_name: '{{ model_name | lower }}'

+ uw config realize -i ../configs/tt05.yaml
model_name: FV3
variable_name: fv3


## UW YAML Custom Tags

`uwtools` provides a number of [custom YAML tags](https://uwtools.readthedocs.io/en/main/sections/user_guide/yaml/tags.html) that can be used to convert string values -- the normal result of evaluating Jinja2 expressions -- to other types, among other purposes.

Returning to examples similar to those seen above:

In [8]:
!realize tt06

tt06.yaml:

nlev: 127
nx: 1440
ny: 721
points: !int '{{ nx * ny * nlev }}'
leadtimes: !list '[{% for n in range(6, 25, 6) %}{{ n }},{% endfor %}]'

+ uw config realize -i ../configs/tt06.yaml
nlev: 127
nx: 1440
ny: 721
points: 131856480
leadtimes:
- 6
- 12
- 18
- 24


This time, `points` is in fact an integer value, and `leadtimes` has been expanded into a proper YAML sequence of integer values.

Among the other tags available are two for working with time values:

In [9]:
!realize tt07

tt07.yaml:

cycle: !datetime 2025-09-24T12
leadtime: !timedelta 6
first_forecast_validtime: !datetime '{{ cycle + leadtime }}'

+ uw config realize -i ../configs/tt07.yaml
cycle: 2025-09-24T12:00:00
leadtime: !timedelta '6:00:00'
first_forecast_validtime: 2025-09-24T18:00:00


Additionally (but not shown here) the `!include` tag can be used to inline another config at a specified point in a config, and `!remove` can be used to eliminate a value from a config when composing multiple configs with `uw config compose` or `uw config realize`. Refer to [the documentation](https://uwtools.readthedocs.io/en/main/sections/user_guide/yaml/tags.html#custom-tags) for more information.

## The UW YAML `env` Custom Filter

`uwtools` defines a custom filter, `env`, providing access to environment variables:

In [10]:
!realize tt08

tt08.yaml:

rundir: '{{ "HOME" | env }}/run'

+ uw config realize -i ../configs/tt08.yaml
rundir: /home/Paul.Madden/run
